In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
from scipy.stats import norm


# Exercise: Risk Analysis for an Investor in the Utilities Sector

An investor is considering investing in assets from the S&P 500, particularly within the Utilities sector, but is concerned about the potential risk of daily losses. To help the investor make an informed decision, you will conduct an analysis to determine which assets in this sector pose the highest and lowest risk, measured by the maximum potential daily loss with a 5% confidence interval, based on historical data.

Using the file componentes_sp500.xlsx (sheet: sp500), focus on the GICS_sector column to identify assets in the Utilities sector and extract their tickers from the Symbol column. Then, download their historical Adjusted Close prices from Yahoo Finance for the period 2010-01-01 to 2020-12-31 and calculate:

- The stock with the highest potential daily loss.

- The stock with the lowest potential daily loss.
  
For the two selected assets, provide the following information:

- Ticker

- Company name

- Subindustry

- Date added to the S&P 500

- Maximum daily loss, calculated with a 5% confidence interval, based on the historical values

- Annual expected return

Store the results for these two assets in a list of dictionaries with the above information as key-value pairs.

Before performing any calculations, drop any assets that have missing values during this period.

After completing this analysis, calculate the potential daily losses for the same two assets using a 99% confidence interval. This time, instead of the historical approach, use a parametric approach.

In [57]:
# Load the data
sp500_components = pd.read_excel('./data/componentes_indices_mod.xlsx', sheet_name='sp500', skiprows=1)

# Filter for Utilities sector
tickers = sp500_components[sp500_components['GICS_Sector'] == 'Utilities']['Symbol'].to_list()

start_date = '2010-01-01'
end_date = '2020-12-31'

# Download Adjusted Close prices from Yahoo Finance
data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

# Drop assets with missing data
data.dropna(axis=1, how='any', inplace=True)

# Calculate daily log returns
data_returns = np.log(data).diff().dropna()

# Calculate 5% historical VaR (Value at Risk) for each asset
var_95 = data_returns.quantile(0.05)

# Find tickers with the highest and lowest potential daily loss
loss_tickers = [var_95.idxmin(), var_95.idxmax()]  # [max_loss_ticker, min_loss_ticker]

# Calculate annual expected return (assuming 252 trading days)
annual_return = data_returns.mean() * 252

# Create a list of dictionaries to store results using f-strings for formatting
results = []
for ticker in loss_tickers:
    asset_info = sp500_components[sp500_components['Symbol'] == ticker].iloc[0]
    results.append({
        'Ticker': ticker,
        'Company name': asset_info['Company_Name'],
        'Subindustry': asset_info['GICS_Sub-Industry'],
        'Date added': asset_info['Date_added'],
        'Maximum daily loss (5%)': f"{100 * var_95[ticker]:.2f}",  # Format as a string with 2 decimals
        'Annual expected return': f"{100 * annual_return[ticker]:.2f}"  # Format as a string with 2 decimals
    })

# Parametric VaR at 99% confidence interval (assuming normal distribution)
mean_returns = data_returns.mean()
std_returns = data_returns.std()

# Calculate parametric VaR for 99% confidence interval
var_99_parametric = mean_returns + norm.ppf(0.01) * std_returns

# Append the parametric VaR (99%) for the two selected assets
for ticker in loss_tickers:
    for result in results:
        if result['Ticker'] == ticker:
            result['Maximum daily loss (99%) Parametric'] = f"{100 * var_99_parametric[ticker]:.2f}"  # Format with 2 decimals

# Display the updated results
results


[*********************100%***********************]  31 of 31 completed

1 Failed download:
['CEG']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2010-01-01 -> 2020-12-31) (Yahoo error = "Data doesn\'t exist for startDate = 1262322000, endDate = 1609390800")')


[{'Ticker': 'NRG',
  'Company name': 'NRG Energy',
  'Subindustry': 'Independent Power Producers & Energy Traders',
  'Date added': '2010-01-29',
  'Maximum daily loss (5%)': '-3.19',
  'Annual expected return': '5.13',
  'Maximum daily loss (99%) Parametric': '-5.18'},
 {'Ticker': 'SO',
  'Company name': 'Southern Company',
  'Subindustry': 'Electric Utilities',
  'Date added': '1957-03-04',
  'Maximum daily loss (5%)': '-1.53',
  'Annual expected return': '10.12',
  'Maximum daily loss (99%) Parametric': '-2.79'}]